<a href="https://colab.research.google.com/github/ttsupra30/QM2/blob/main/QM_updated_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Method Overview
**Objective**: This study examines the relationship between gambling expenditure and labour market conditions in Australia, focusing on unemployment rates across eight regions and at the national level. The analysis proceeds in two stages:

**Panel regression** to identify robust **correlations** while controlling for unobserved regional heterogeneity.

**Granger causality analysis** to examine lead–lag dynamics and temporal **causation**.

This two-step approach allows the study to separate contemporaneous association from dynamic predictive causality.

# 1. Data
**1.1 Unit of Analysis**

Cross-sectional unit: Australian regions $i = ACT, NSW, \dots, WA$

Time unit: annual observations $t=1,\dots,T$

Balanced panel: each region observed over the same time span

**1.2 Key Variables**

*  Dependent or independent relies on the test direction

*  $Gambling_t^{(i)}$ : real per-capita gambling expenditure $(\$)$

*  $Unemp_t^{(i)}$ : regional unemployment rate $(\%)$

**1.3 Transformations**

$Gambling_t^{(i)} → ln(Gambling_t^{(i)})$

**1.4 Control variables ($X_t^{(i)}$):**

Quarterly Population Estimates (Persons)

Wage price index

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
!pip install linearmodels
from linearmodels.panel import PanelOLS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 11.4 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# Read data
df = pd.read_csv('QM2.csv')
df.head()

In [ ]:
# Convert year columns
df.dropna(subset=['Year'], inplace=True)
df['year'] = df['Year'].str[-2:].astype(int) + 2000
df.loc[df['year'] > 2025, 'year'] -= 100
df

In [ ]:
# Relabel the columns
STATE = "State"
YEAR  = "year"
GAMB  = "Real per capita total gambling expenditure value ($)"
UNEMP = "Unemployment rate (%)"
WPI   = "Wage price index"
POP   = "Final consumption expenditure minus net loss from gambling ( Millions $)"

# Keep only necessary columns
df = df[[STATE, YEAR, GAMB, UNEMP, WPI, POP]].copy()

# Ensure numeric data of all variables
df[[GAMB, UNEMP, WPI, POP]] = df[[GAMB, UNEMP, WPI, POP]].apply(
    pd.to_numeric, errors="coerce"
)

# Log gambling (must be > 0)
df = df[df[GAMB] > 0]
df["log_gambling"] = np.log(df[GAMB])

# Log controls (must be > 0)
df = df[df[POP] > 0]
df["log_pop"] = np.log(df[POP])

# Drop missing and set panel index
df = df.dropna(subset=[STATE, YEAR, "log_gambling", UNEMP, WPI, "log_pop"])
df = df.set_index([STATE, YEAR]).sort_index()

df

# 3. Models
# 3.1 Time-series OLS + Panel Regression
**3.1.A Model A: Gambling leads unemployment**

*   Test of gambling → later unemployment




**Model A1: National time-series OLS (Australia as a whole)**

$$
Unemp_t
=
\alpha
+\sum_{k=0}^{p}\beta_k\,\ln(Gambling_{t-k})
+\delta_2 \ln(Pop_t)
+\delta_3 WPI_t
+\varepsilon_t
$$

**Model A2: Single-region time-series OLS** (e.g. NSW only)

* Same equation - just applied to one region.

$$
Unemp_{i,t}
=
\alpha
+\sum_{k=0}^{p}\beta_k\,\ln(Gambling_{i,t-k})
+\delta_2 \ln(Pop_{i,t})
+\delta_3 WPI_{i,t}
+\varepsilon_{i,t}
$$

\begin{align}
H_0 &: \beta_0 = \beta_1 = \cdots = \beta_p = 0
&& \text{(Gambling expenditure has no association with unemployment)} \\
H_1 &: \exists \; k \in \{0,\dots,p\} \text{ such that } \beta_k \neq 0
&& \text{(Gambling expenditure is associated with unemployment at some lag)}
\end{align}

Both two times series OLS test for “Does gambling correlate with later unemployment within this specific region over time?”

In [ ]:
def ts_ols_modelA(ts_df, *, year_col=YEAR, p=1):

    d = (
        ts_df[[year_col, UNEMP, "log_gambling", "log_pop", WPI]]
        .dropna()
        .sort_values(year_col)
        .copy()
    )

    # Lag gambling INCLUDING contemporaneous term (k = 0)
    for k in range(0, p + 1):
        d[f"g_L{k}"] = d["log_gambling"].shift(k)

    rhs = (
        [f"g_L{k}" for k in range(0, p + 1)]
        + [ "log_pop", WPI]
    )

    d = d.dropna(subset=[UNEMP] + rhs)

    y = d[UNEMP].astype(float)
    X = sm.add_constant(d[rhs].astype(float))

    res = sm.OLS(y, X).fit(
        cov_type="HAC",
        cov_kwds={"maxlags": p}
    )

    return res, d


In [ ]:
#Call National level
df_flat = df.reset_index()

nat = (
    df_flat
    .groupby(YEAR)[[UNEMP, "log_gambling", "log_pop", WPI]]
    .mean()
    .reset_index()
)

res_nat, used_nat = ts_ols_modelA(nat, year_col=YEAR, p=5)
print(res_nat.summary())


In [ ]:
# Call National level Exclude Northern Territory
df_flat_noNT = df_flat[df_flat["State"] != "NT"].copy()
# National series excluding NT (average across remaining states)
nat_noNT = (
    df_flat_noNT
    .groupby(YEAR)[[UNEMP, "log_gambling","log_pop", WPI]]
    .mean()
    .reset_index()
)

res_nat_noNT, used_nat_noNT = ts_ols_modelA(
    nat_noNT,
    year_col=YEAR,
    p=5
)

print(res_nat_noNT.summary())


In [ ]:
#Call Regional level
region_nsw = df_flat[df_flat["State"] == "NSW"][
    [YEAR, UNEMP, "log_gambling", "log_pop", WPI]
]

res_nsw, used_nsw = ts_ols_modelA(region_nsw, year_col=YEAR, p=3)
print(res_nsw.summary())

In [ ]:
# MAKE ONE RESULT TABLE (National, 8 states, National excl NT) ======

from statsmodels.iolib.summary2 import summary_col
import numpy as np
import pandas as pd

# ---------- helpers ----------
def cumulative_gambling_effect(res, p):
    return sum(float(res.params.get(f"g_L{k}", 0.0)) for k in range(0, p + 1))

def wald_pvalue_gambling_raw(res, p):
    terms = [f"g_L{k}" for k in range(0, p + 1)]
    hypothesis = " = ".join(terms) + " = 0"
    wt = res.wald_test(hypothesis, scalar=True)  # scalar=True avoids FutureWarning
    return float(wt.pvalue)

def format_p(pval, decimals=6):
    # raw numeric p-value, formatted consistently; if extremely tiny, show scientific
    if pval == 0.0:
        return "0"
    if pval < 10**(-decimals):
        return f"{pval:.2e}"
    return f"{pval:.{decimals}f}"

# ---------- settings ----------
P = 5   # lag length used in ts_ols_modelA (must match your estimation)
EXCL = "NT"      # exclude NT for the robustness national series

# ---------- flatten panel ----------
df_flat = df.reset_index()

# ---------- National (All) ----------
nat_all = (
    df_flat.groupby(YEAR)[[UNEMP, "log_gambling", "log_pop", WPI]]
    .mean()
    .reset_index()
)
model_nat_all, _ = ts_ols_modelA(nat_all, year_col=YEAR, p=P)

# ---------- State models ----------
state_models, state_names = [], []
for st in sorted(df_flat["State"].dropna().unique()):
    sub = df_flat[df_flat["State"] == st][
        [YEAR, UNEMP, "log_gambling",  "log_pop", WPI]
    ]
    res, _ = ts_ols_modelA(sub, year_col=YEAR, p=P)
    state_models.append(res)
    state_names.append(st)

# ---------- National (exclude NT) ----------
nat_excl = (
    df_flat[df_flat["State"] != EXCL]
    .groupby(YEAR)[[UNEMP, "log_gambling",  "log_pop", WPI]]
    .mean()
    .reset_index()
)
model_nat_excl, _ = ts_ols_modelA(nat_excl, year_col=YEAR, p=P)

# ---------- build summary_col table ----------
models = [model_nat_all] + state_models + [model_nat_excl]
names  = ["National (All)"] + state_names + [f"National (excl {EXCL})"]

regressor_order = (
    ["const"]
    + [f"g_L{k}" for k in range(0, P + 1)]
    + [ "log_pop", WPI]
)

# Keep info_dict minimal to avoid duplicated lines
table = summary_col(
    models,
    stars=True,
    float_format="%0.3f",
    model_names=names,
    regressor_order=regressor_order,
    info_dict={
        "N": lambda x: f"{int(x.nobs)}"
    },
)

# ---------- append extra rows (aligned as normal rows) ----------
df_table = table.tables[0]

cum_vals = [f"{cumulative_gambling_effect(m, P):.3f}" for m in models]
wald_vals = [format_p(wald_pvalue_gambling_raw(m, P), decimals=6) for m in models]

df_table.loc[f"Cumulative gambling effect (sum g_L0..g_L{P})"] = cum_vals
df_table.loc[f"Wald p-value (H0: g_L0..g_L{P} = 0)"] = wald_vals

# ---------- print in a “parallel” aligned way (avoid wrapping) ----------
with pd.option_context(
    "display.width", 4000,
    "display.max_columns", None,
    "display.max_colwidth", None
):
    print(df_table.to_string())


In [ ]:
# ---------- Sensitivity test over lag length p ----------

P_MAX = 8   # or whatever maximum lag you want

rows = []

for p in range(0, P_MAX + 1):

    # run NATIONAL model (example: national average series)
    res, used = ts_ols_modelA(nat_all, year_col=YEAR, p=p)

    # cumulative effect
    cum_eff = cumulative_gambling_effect(res, p)

    # Wald test
    terms = [f"g_L{k}" for k in range(0, p + 1)]
    hypothesis = " = ".join(terms) + " = 0"
    wt = res.wald_test(hypothesis, scalar=True)

    rows.append({
        "k": p,
        "Cumulative gambling effect": cum_eff,
        "Wald stat": float(wt.statistic),
        "Wald p-value": float(wt.pvalue),
        "N": int(res.nobs),
    })

# build table
sens_table = pd.DataFrame(rows).set_index("p")

# pretty print
with pd.option_context(
    "display.float_format", "{:.6g}".format,
    "display.width", 2000
):
    print(sens_table)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

# ----------------------------
# ASSUMES you already have:
# df_flat with columns: [STATE, YEAR, UNEMP, log_gambling, log_pop, WPI]
# and constants:
# STATE="State", YEAR="year", UNEMP="Unemployment rate (%)", WPI="Wage price index"
# ----------------------------

def ts_ols_modelA(ts_df, *, year_col, p):
    """
    Unemployment_t ~ g_L0..g_Lp + log_pop + WPI
    HAC SEs with maxlags=p
    Returns (res, d_used) where d_used includes the lag columns.
    """
    d = (
        ts_df[[year_col, UNEMP, "log_gambling", "log_pop", WPI]]
        .dropna()
        .sort_values(year_col)
        .copy()
    )

    for k in range(0, p + 1):
        d[f"g_L{k}"] = d["log_gambling"].shift(k)

    rhs = [f"g_L{k}" for k in range(0, p + 1)] + ["log_pop", WPI]
    d = d.dropna(subset=[UNEMP] + rhs)

    y = d[UNEMP].astype(float)
    X = sm.add_constant(d[rhs].astype(float))

    res = sm.OLS(y, X).fit(cov_type="HAC", cov_kwds={"maxlags": p})
    return res, d


def plot_actual_vs_fitted_modelA(ts_df, label, *, year_col, p, fig_number):
    """
    Plots Actual vs Fitted unemployment from Model A
    """
    res, d_used = ts_ols_modelA(ts_df, year_col=year_col, p=p)

    rhs = [f"g_L{k}" for k in range(0, p + 1)] + ["log_pop", WPI]
    X_used = sm.add_constant(d_used[rhs].astype(float))
    fitted = res.predict(X_used)

    plt.figure(figsize=(9, 4.5))
    plt.plot(
        d_used[year_col],
        d_used[UNEMP],
        marker="o",
        label="Actual unemployment"
    )
    plt.plot(
        d_used[year_col],
        fitted,
        marker="o",
        linestyle="--",
        label="Fitted (OLS model)"
    )

    plt.title(
        f"Figure {fig_number}: {label} : OLS Model (k={p}), gambling → unemployment"
    )
    plt.xlabel("Year")
    plt.ylabel("Unemployment rate (%)")
    plt.legend()
    plt.tight_layout()
    plt.show()

    return res


# ---------- RUN EVERYTHING ----------
def run_all_plots_modelA(df_flat, *, p=2, section=1):
    fig_counter = 1

    # National (all states)
    nat_all = (
        df_flat
        .groupby(YEAR)[[UNEMP, "log_gambling", "log_pop", WPI]]
        .mean()
        .reset_index()
    )

    plot_actual_vs_fitted_modelA(
        nat_all,
        "National",
        year_col=YEAR,
        p=p,
        fig_number=f"{section}.{fig_counter}"
    )
    fig_counter += 1

    # State-level plots
    states = sorted(df_flat[STATE].dropna().unique())
    for st in states:
        st_df = df_flat[df_flat[STATE] == st][
            [YEAR, UNEMP, "log_gambling", "log_pop", WPI]
        ].copy()

        plot_actual_vs_fitted_modelA(
            st_df,
            st,
            year_col=YEAR,
            p=p,
            fig_number=f"{section}.{fig_counter}"
        )
        fig_counter += 1


# Example call
df_flat = df.reset_index() # Re-initialize df_flat
run_all_plots_modelA(df_flat, p=2, section=1)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

# ----------------------------
# ASSUMES you already have:
# df_flat with columns: [STATE, YEAR, UNEMP, log_gambling, log_pop, WPI]
# ----------------------------

def ts_ols_modelB(ts_df, *, year_col, p):
    """
    log_gambling_t ~ u_L0..u_Lp + log_pop + WPI
    HAC SEs with maxlags=p
    Returns (res, d_used) where d_used includes the lag columns.
    """
    d = (
        ts_df[[year_col, UNEMP, "log_gambling", "log_pop", WPI]]
        .dropna()
        .sort_values(year_col)
        .copy()
    )

    for k in range(0, p + 1):
        d[f"u_L{k}"] = d[UNEMP].shift(k)

    rhs = [f"u_L{k}" for k in range(0, p + 1)] + ["log_pop", WPI]
    d = d.dropna(subset=["log_gambling"] + rhs)

    y = d["log_gambling"].astype(float)
    X = sm.add_constant(d[rhs].astype(float))

    res = sm.OLS(y, X).fit(cov_type="HAC", cov_kwds={"maxlags": p})
    return res, d


def plot_actual_vs_fitted_modelB(ts_df, label, *, year_col, p):
    """
    Plots Actual log gambling vs Fitted log gambling from Model B.
    """
    res, d_used = ts_ols_modelB(ts_df, year_col=year_col, p=p)

    rhs = [f"u_L{k}" for k in range(0, p + 1)] + ["log_pop", WPI]
    X_used = sm.add_constant(d_used[rhs].astype(float))
    fitted = res.predict(X_used)

    plt.figure(figsize=(9, 4.5))
    plt.plot(d_used[year_col], d_used["log_gambling"], marker="o", label="Actual log gambling")
    plt.plot(d_used[year_col], fitted, marker="o", linestyle="--", label="Fitted (OLS Model)")
    plt.title(f"{label} : OLS Model (k={p}) : unemployment → gambling")
    plt.xlabel("Year")
    plt.ylabel("Log gambling expenditure (per capita)")
    plt.legend()
    plt.tight_layout()
    plt.show()

    return res


# ----------------------------
# RUN EVERYTHING
# ----------------------------
def run_all_plots_modelB(df_flat, *, p=2, exclude_nt=True):
    # National (All)
    nat_all = (
        df_flat.groupby(YEAR)[[UNEMP, "log_gambling", "log_pop", WPI]]
        .mean()
        .reset_index()
    )
    plot_actual_vs_fitted_modelB(nat_all, "National (All)", year_col=YEAR, p=p)

    # National (excl NT)
    if exclude_nt:
        nat_excl = (
            df_flat[df_flat[STATE] != "NT"]
            .groupby(YEAR)[[UNEMP, "log_gambling", "log_pop", WPI]]
            .mean()
            .reset_index()
        )
        plot_actual_vs_fitted_modelB(nat_excl, "National (excl NT)", year_col=YEAR, p=p)

    # States
    states = sorted(df_flat[STATE].dropna().unique())
    for st in states:
        st_df = df_flat[df_flat[STATE] == st][[YEAR, UNEMP, "log_gambling", "log_pop", WPI]].copy()
        plot_actual_vs_fitted_modelB(st_df, st, year_col=YEAR, p=p)


# Example call:
# run_all_plots_modelB(df_flat, p=2, exclude_nt=True)
run_all_plots_modelB(df_flat, p=2)

**Intepret the Coefficient**

Note: Here I deliberately set $k$ starting from $0$, so that when $k=0$ we capture the contemporaneous effect, that is, how unemployment and gambling move within the same year.

Updated: I've included the cumulative gambling effect row and Wald p-value to help the analysis.

**Model A3: 8-region panel** (main regional analysis)

$$
Unemp_t
=
\alpha
+\sum_{k=0}^{p}\beta_k\,\ln(Gambling_{t-k})
+\delta_2 \ln(Pop_t)
+\delta_3 WPI_t
+\mu_i
+\lambda_t
+\eta_{i,t}
$$

\begin{align}
H_0 &: \sum_{k=0}^{p}\theta_k = 0,
&& k = 0,1,\dots,p
&& \text{(No overall association over the lag window)} \\
H_1 &: \sum_{k=0}^{p}\theta_k \neq 0,
&& k = 0,1,\dots,p
&& \text{(Overall association over the lag window)}
\end{align}


**Note:** We only use panel regression when we have both **cross-sectional** and **time-series** data. In this case, it only works when we include all 8 regions together with their time-series data.

In the previous section, I use time-series OLS because we do not use the cross-sectional dimension and instead examine one region at a time.


Panel model tells “Within regions, does gambling lead unemployment **after accounting for regional differences and national shocks?**”

**The error terms**

**$\bf 1. \; \mu^{(i)}$ :** Region fixed effects $:=$ everything about that region that does NOT change over time

“Controls for things that make $NSW$ permanently different from $WA$.”

Examples: Cultural attitudes toward gambling, Long-run industry structure (e.g. mining-heavy $WA$), Historical labour-market characteristics, etc.

These factors differ across regions but are constant over time within each region.

**$\bf 2. \; \lambda_t$ :** Year fixed effects $:=$ everything that affects all regions in year

“Controls for things that affect all regions in a given year, like COVID.”

Examples: National recessions / booms, COVID, Interest-rate cycles, etc.

So if unemployment and gambling both spike in 2020: that variation is absorbed by 𝜆 not attributed to gambling causing unemployment

**$\bf 3. \; \epsilon_t^{(i)}$ :** Idiosyncratic shocks $:=$ unexpected region-year noise

“Everything else we can't explain.”

Examples: A local factory closure, A temporary regional event, Measurement error, etc.

In [ ]:
def panel_fe_modelA(df_panel, *, p=1):
    d = df_panel.copy()

    # Ensure MultiIndex (STATE, YEAR)
    if not isinstance(d.index, pd.MultiIndex):
        d = d.set_index([STATE, YEAR]).sort_index()
    else:
        d = d.sort_index()

    # Required columns
    required = [UNEMP, "log_gambling", "log_pop", WPI]
    missing = [c for c in required if c not in d.columns]
    if missing:
        raise KeyError(f"Missing columns in df_panel: {missing}")

    # Build gambling lags within state: k = 0..p
    for k in range(0, p + 1):
        d[f"g_L{k}"] = d.groupby(level=0)["log_gambling"].shift(k)

    # Regressors: gambling lags + controls
    g_terms = [f"g_L{k}" for k in range(0, p + 1)]
    rhs = g_terms + ["log_pop", WPI]

    # Drop missing due to lags
    d = d[[UNEMP] + rhs].dropna()

    y = d[UNEMP].astype(float)
    X = d[rhs].astype(float)

    mod = PanelOLS(y, X, entity_effects=True, time_effects=True)
    res = mod.fit(cov_type="clustered", cluster_entity=True)

    # Cumulative effect: sum_{k=0..p} beta_k
    cum_effect = float(res.params[g_terms].sum())

    # Joint Wald test: H0: g_L0 = ... = g_Lp = 0
    param_names = list(res.params.index)
    K = len(param_names)
    R = np.zeros((len(g_terms), K))
    for i, term in enumerate(g_terms):
        R[i, param_names.index(term)] = 1.0
    q = np.zeros(len(g_terms))

    wald = res.wald_test(R, q)
    wald_stat = float(wald.stat)
    wald_pval = float(wald.pval)

    extra = pd.DataFrame(
        {
            "Cumulative gambling effect": [cum_effect],
            "Wald stat (H0: g_L0..g_Lp = 0)": [wald_stat],
            "Wald p-value (H0: g_L0..g_Lp = 0)": [wald_pval],
            "N": [int(res.nobs)],
            "p": [int(p)],
        }
    )

    return res, extra



In [ ]:
# ---- HOW TO CALL THE PANEL (choose one p) ----
# Example: run the panel with p = 3
res_panel, extra_panel = panel_fe_modelA(df, p=3)

# regression output
print(res_panel.summary)

# diagnostics as separate rows (vertical)
print(extra_panel.T.rename(columns={0: "value"}))

In [ ]:
#PART 2 — Sensitivity test over multiple p
def sensitivity_test_panelA(df_panel, p_list=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]):
    rows = []

    for p in p_list:
        _, extra = panel_fe_modelA(df_panel, p=p)
        rows.append(extra)

    out = pd.concat(rows, ignore_index=True).set_index("p")
    return out

In [ ]:
# ---- HOW TO CALL THE SENSITIVITY TEST ----
# Example: test p = 1..5
sens = sensitivity_test_panelA(df, p_list=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])
sens

**3.1.B Model B: Gambling lags unemployment**

Test of unemployment → later gambling




This part should be really easy to understand if you understand 3.1.A, we are just switching the dependent and independent variables.
Therefore, I'll simplify the explanation ;)



**Model B1 and B2: Time-series OLS run at National and Regional level**

$$
\ln(Gambling_{i,t})
=
\alpha
+\sum_{k=0}^{p}\theta_k\,Unemp_{i,t-k}
+\delta_2 \ln(Pop_{i,t})
+\delta_3 WPI_{i,t}
+\varepsilon_{i,t}
$$

\begin{align}
H_0 &: \beta_0 = \beta_1 = \cdots = \beta_p = 0,
\qquad k \in \{0,1,\dots,p\}
&& \text{(Gambling expenditure has no association with unemployment)} \\
H_1 &: \exists\, k \in \{0,1,\dots,p\} \text{ such that } \beta_k \neq 0
&& \text{(Gambling expenditure is associated with unemployment over the lag window)}
\end{align}


In [ ]:
# Model B (time-series OLS): log_gambling_t on lags of unemployment + controls (log_awe, log_pop, WPI)

def ts_ols_modelB(ts_df, *, year_col=YEAR, p=1):
    """
    log_gambling_t ~ u_{t}, u_{t-1}, ..., u_{t-p} + log_awe_t + log_pop_t + WPI_t
    HAC (Newey–West) SEs with maxlags = p
    """

    d = (
        ts_df[[year_col, UNEMP, "log_gambling", "log_pop", WPI]]
        .dropna()
        .sort_values(year_col)
        .copy()
    )

    # Lag unemployment: k = 0..p
    for k in range(0, p + 1):
        d[f"u_L{k}"] = d[UNEMP].shift(k)

    rhs = [f"u_L{k}" for k in range(0, p + 1)] + ["log_pop", WPI]
    d = d.dropna(subset=["log_gambling"] + rhs)

    y = d["log_gambling"].astype(float)
    X = sm.add_constant(d[rhs].astype(float))

    res = sm.OLS(y, X).fit(cov_type="HAC", cov_kwds={"maxlags": p})
    return res, d


In [ ]:
# Call Model B1
# National
df_flat = df.reset_index()
nat = df_flat.groupby(YEAR)[[UNEMP, "log_gambling", "log_pop", WPI]].mean().reset_index()

res_nat_B, used_nat_B = ts_ols_modelB(nat, year_col=YEAR, p=5)# change p here
print(res_nat_B.summary())

In [ ]:
#Call Model B2
# Regional
region = df_flat[df_flat["State"] == "NSW"][[YEAR, UNEMP, "log_gambling", "log_pop", WPI]] # change region here

res_nsw_B, used_nsw_B = ts_ols_modelB(region, year_col=YEAR, p=3)# change p here
print(res_nsw_B.summary())

In [ ]:
# ========= MODEL B RESULTS TABLE (National + 8 states + National excl NT)


# ---- helpers ----
def cumulative_unemp_effect(res, p):
    return sum(float(res.params.get(f"u_L{k}", 0.0)) for k in range(0, p + 1))

def wald_pvalue_unemp_block(res, p):
    terms = [f"u_L{k}" for k in range(0, p + 1)]
    hypothesis = " = ".join(terms) + " = 0"
    wt = res.wald_test(hypothesis, scalar=True)  # avoids FutureWarning
    return float(wt.pvalue)

def fmt_p(pval, decimals=6):
    # raw p-value, readable (scientific if very small)
    if pval == 0.0:
        return "0"
    if pval < 10**(-decimals):
        return f"{pval:.2e}"
    return f"{pval:.{decimals}f}"

# ---- settings ----
P = 5       # set lag length here (e.g., 3 if you want u_L0..u_L3)
EXCL = "NT"    # exclude NT for robustness national series

# ---- flatten ----
df_flat = df.reset_index()

# ---- National (All) ----
nat_all = (
    df_flat.groupby(YEAR)[[UNEMP, "log_gambling",  "log_pop", WPI]]
    .mean()
    .reset_index()
)
model_nat_all, _ = ts_ols_modelB(nat_all, year_col=YEAR, p=P)

# ---- Each state ----
state_models, state_names = [], []
for st in sorted(df_flat["State"].dropna().unique()):
    sub = df_flat[df_flat["State"] == st][
        [YEAR, UNEMP, "log_gambling",  "log_pop", WPI]
    ]
    res, _ = ts_ols_modelB(sub, year_col=YEAR, p=P)
    state_models.append(res)
    state_names.append(st)

# ---- National (exclude NT) ----
nat_excl = (
    df_flat[df_flat["State"] != EXCL]
    .groupby(YEAR)[[UNEMP, "log_gambling",  "log_pop", WPI]]
    .mean()
    .reset_index()
)
model_nat_excl, _ = ts_ols_modelB(nat_excl, year_col=YEAR, p=P)

# ---- Build table ----
models = [model_nat_all] + state_models + [model_nat_excl]
names  = ["National (All)"] + state_names + [f"National (excl {EXCL})"]

regressor_order = (
    ["const"]
    + [f"u_L{k}" for k in range(0, P + 1)]
    + [ "log_pop", WPI]
)

table = summary_col(
    models,
    stars=True,
    float_format="%0.3f",
    model_names=names,
    regressor_order=regressor_order,
    info_dict={
        "R-squared": lambda x: f"{x.rsquared:.3f}",
        "R-squared Adj.": lambda x: f"{x.rsquared_adj:.3f}",
        "N": lambda x: f"{int(x.nobs)}",
    },
)

# ---- Append extra rows ----
df_table = table.tables[0]

cum_vals  = [f"{cumulative_unemp_effect(m, P):.3f}" for m in models]
wald_pvals = [fmt_p(wald_pvalue_unemp_block(m, P), decimals=6) for m in models]

df_table.loc[f"Cumulative unemployment effect (sum u_L0..u_L{P})"] = cum_vals
df_table.loc[f"Wald p-value (H0: u_L0..u_L{P} = 0)"] = wald_pvals

# ---- Print aligned ----
with pd.option_context("display.width", 4000, "display.max_columns", None, "display.max_colwidth", None):
    print(df_table.to_string())


In [ ]:
# ---------- Sensitivity test over lag length p ----------

P_MAX = 10   # or whatever maximum lag you want

rows = []

for p in range(0, P_MAX + 1):

    # run NATIONAL model (example: national average series)
    res, used = ts_ols_modelB(nat_all, year_col=YEAR, p=p)

    # cumulative effect (using the correct function for unemployment lags)
    cum_eff = cumulative_unemp_effect(res, p)

    # Wald test (using the correct function for unemployment lags)
    # The function wald_pvalue_unemp_block expects the model results and p
    wald_p_val = wald_pvalue_unemp_block(res, p)

    # To get the statistic, we need to manually reconstruct the Wald test as wald_pvalue_unemp_block only returns p-value
    terms = [f"u_L{k}" for k in range(0, p + 1)]
    hypothesis = " = ".join(terms) + " = 0"
    wt = res.wald_test(hypothesis, scalar=True)
    wald_stat = float(wt.statistic)

    rows.append({
        "p": p,
        "Cumulative unemployment effect": cum_eff,
        "Wald stat": wald_stat,
        "Wald p-value": wald_p_val,
        "N": int(res.nobs),
    })

# build table
sens_table = pd.DataFrame(rows).set_index("p")
sens_table.index.name = "k"

# pretty print
with pd.option_context(
    "display.float_format", lambda x: f"{x:.6g}",
    "display.width", 2000
):
    print(sens_table)

**Model B3: 8-region panel**

$$
\ln(Gambling_{i,t})
=
\alpha
+\sum_{k=0}^{p}\theta_k\,Unemp_{i,t-k}
+\delta_2 \ln(Pop_{i,t})
+\delta_3 WPI_{i,t}
+\mu_i
+\lambda_t
+\eta_{i,t}
$$

\begin{align}
H_0 &: \sum_{k=0}^{p}\theta_k = 0,
&& k = 0,1,\dots,p
&& \text{(No overall association over the lag window)} \\
H_1 &: \sum_{k=0}^{p}\theta_k \neq 0,
&& k = 0,1,\dots,p
&& \text{(Overall association over the lag window)}
\end{align}

In [ ]:
# Model B (panel FE): log_gambling on lags of unemployment + controls (log_awe, log_pop, WPI)

def panel_modelB(df_panel, *, p=1):


    d = df_panel.copy()

    # Ensure MultiIndex (STATE, YEAR)
    if not isinstance(d.index, pd.MultiIndex):
        d = d.set_index([STATE, YEAR]).sort_index()
    else:
        d = d.sort_index()

    # Lags of unemployment within state: k = 0..p
    for k in range(0, p + 1):
        d[f"u_L{k}"] = d.groupby(level=0)[UNEMP].shift(k)

    # Regressors: unemployment lags + controls
    rhs = [f"u_L{k}" for k in range(0, p + 1)] + [ "log_pop", WPI]

    # Keep complete cases
    d = d.dropna(subset=["log_gambling"] + rhs)

    y = d["log_gambling"].astype(float)
    X = d[rhs].astype(float)

    mod = PanelOLS(y, X, entity_effects=True, time_effects=True)
    return mod.fit(cov_type="clustered", cluster_entity=True)


In [ ]:
# call panel
resB_panel = panel_modelB(df, p=5)
print(resB_panel.summary)

In [ ]:
# =========================
# PART 1 — Panel FE Model B (reverse direction)
#   log_gambling_{i,t} on unemployment lags + controls
#   + cumulative effect + joint Wald test
# =========================

import numpy as np
import pandas as pd
from linearmodels.panel import PanelOLS

def panel_fe_modelB(df_panel, *, p=1):
    d = df_panel.copy()

    # Ensure MultiIndex (STATE, YEAR)
    if not isinstance(d.index, pd.MultiIndex):
        d = d.set_index([STATE, YEAR]).sort_index()
    else:
        d = d.sort_index()

    # Required columns
    required = ["log_gambling", UNEMP, "log_pop", WPI]
    missing = [c for c in required if c not in d.columns]
    if missing:
        raise KeyError(f"Missing columns in df_panel: {missing}")

    # Lags of unemployment within state: k = 0..p
    for k in range(0, p + 1):
        d[f"u_L{k}"] = d.groupby(level=0)[UNEMP].shift(k)

    # Regressors: unemployment lags + controls
    u_terms = [f"u_L{k}" for k in range(0, p + 1)]
    rhs = u_terms + ["log_pop", WPI]

    # Keep complete cases (lags induce N drop)
    d = d[["log_gambling"] + rhs].dropna()

    y = d["log_gambling"].astype(float)
    X = d[rhs].astype(float)

    mod = PanelOLS(y, X, entity_effects=True, time_effects=True)
    res = mod.fit(cov_type="clustered", cluster_entity=True)

    # ---- Cumulative effect: sum_{k=0..p} theta_k ----
    cum_effect = float(res.params[u_terms].sum())

    # ---- Joint Wald test: H0: u_L0 = ... = u_Lp = 0 ----
    param_names = list(res.params.index)
    K = len(param_names)
    R = np.zeros((len(u_terms), K))
    for i, term in enumerate(u_terms):
        R[i, param_names.index(term)] = 1.0
    q = np.zeros(len(u_terms))

    wald = res.wald_test(R, q)
    wald_stat = float(wald.stat)
    wald_pval = float(wald.pval)

    extra = pd.DataFrame(
        {
            "Cumulative unemployment effect": [cum_effect],
            "Wald stat (H0: u_L0..u_Lp = 0)": [wald_stat],
            "Wald p-value (H0: u_L0..u_Lp = 0)": [wald_pval],
            "N": [int(res.nobs)],
            "p": [int(p)],
        }
    )

    return res, extra

In [ ]:
# ---- HOW TO CALL THE PANEL (choose one p) ----
# Example: run Model B panel with p = 3
resB_panel, extraB_panel = panel_fe_modelB(df, p=3)

print(resB_panel.summary)
print(extraB_panel.T.rename(columns={0: "value"}))

In [ ]:
# =========================
# PART 2 — Sensitivity test for Panel Model B over multiple p
# =========================

def sensitivity_test_panelB(df_panel, p_list=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14)):
    rows = []
    for p in p_list:
        _, extra = panel_fe_modelB(df_panel, p=p)
        rows.append(extra)

    out = pd.concat(rows, ignore_index=True).set_index("p")
    return out

In [ ]:
# ---- HOW TO CALL THE SENSITIVITY TEST ----
# Example: test p = 1..5
sensB = sensitivity_test_panelB(df, p_list=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])
sensB

All of the models in Section 3.1 only provide information about **correlation**. To further examine whether a **causal** relationship exists, we employ Granger causality analysis.

What “Granger causality” actually means is not philosophical causality. Instead, it asks a specific and testable question:

Do past values of gambling improve our ability to predict unemployment, over and above unemployment’s own past values?

If yes, gambling is said to Granger-cause unemployment.

But the good news is that Granger causality is much simpler than Section 3.1, where we used combined OLS and panel regression. For the panel analysis, we use the same equation and model structure for both national-level and regional-level data, and we simply switch the dependent and independent variables.

# 3.2 Conditional Granger causality tests

3.2.C Model C: Gambling Granger-causes unemployment
* supports gambling leads unemployment

$$
Unemp_t
=
\alpha
+\sum_{k=1}^{p}\gamma_k\,Unemp_{t-k}
+\sum_{k=1}^{p}\beta_k\,\ln(Gambling_{t-k})
+\delta_2 \ln(Pop_t)
+\delta_3 WPI_t
+\epsilon_t
$$

**Hypothesis**:


\begin{align}
H_0 &: \beta_1 = \beta_2 = \cdots = \beta_p = 0
&& \text{(Lagged gambling has no predictive content for unemployment)} \\
H_1 &: \exists\, k \in \{1,\dots,p\} \text{ such that } \beta_k \neq 0
&& \text{(Lagged gambling has predictive content for unemployment)}
\end{align}

In [ ]:
def granger_gambling_to_unemp_joint(
    ts_df,
    *,
    year_col=YEAR,
    maxlag=4,
    controls=("log_pop", WPI)
):
    def stars(p):
        if p < 0.01:
            return "***"
        if p < 0.05:
            return "**"
        if p < 0.1:
            return "*"
        return ""

    cols = [year_col, UNEMP, "log_gambling", *controls]
    d = ts_df[cols].dropna().sort_values(year_col).copy()

    rows = []

    for L in range(1, maxlag + 1):
        tmp = d.copy()

        # build lags
        for k in range(1, L + 1):
            tmp[f"u_L{k}"] = tmp[UNEMP].shift(k)
            tmp[f"g_L{k}"] = tmp["log_gambling"].shift(k)

        tmp = tmp.dropna()

        y = tmp[UNEMP].astype(float)

        X = sm.add_constant(
            tmp[[f"u_L{k}" for k in range(1, L + 1)] +
                [f"g_L{k}" for k in range(1, L + 1)] +
                list(controls)]
            .astype(float)
        )

        res = sm.OLS(y, X).fit()

        # joint F-test: all gambling lags = 0
        param_names = list(X.columns) # Re-adding this line
        # Fix: R should have L+1 rows to test g_L0, ..., g_LL
        # Note: Granger causality typically tests only for lagged effects (k=1 to L), not contemporaneous (k=0).
        # The current R definition and loop range for k (1 to L) correctly reflect this.
        R = np.zeros((L, len(param_names))) # Changed back to L rows because `k` goes from 1 to L, so we test L parameters.
        for i, k in enumerate(range(1, L + 1)): # k starts from 1, so indices in R will be 0 to L-1
            R[i, param_names.index(f"g_L{k}")] = 1.0

        ftest = res.f_test(R)

        # cumulative direction
        cum_coef = sum(res.params[f"g_L{k}"] for k in range(1, L + 1))

        rows.append({
            "Lag": L,
            "Cumulative gambling effect": cum_coef,
            "Direction": "Positive" if cum_coef > 0 else "Negative",
            "F_stat": float(ftest.fvalue),
            "p_value": float(ftest.pvalue),
            "N": int(res.nobs)
        })

    return pd.DataFrame(rows)

In [ ]:
#national
gc_nat = granger_gambling_to_unemp_joint(
    nat_all,
    year_col=YEAR,
    maxlag=7
)
print(gc_nat)


In [ ]:
#regional
# Corrected: Use .isin() for selecting multiple states
region = df_flat[df_flat["State"] == "QLD"][
    [YEAR, UNEMP, "log_gambling", "log_pop", WPI]
]

gc_nsw = granger_gambling_to_unemp_joint(
    region,
    year_col=YEAR,
    maxlag=5
)
print(gc_nsw)

3.2.D Model D2: Unemployment Granger-causes gambling
* supports gambling lags unemployment

$$
\ln(Gambling_t)
=
\alpha
+\sum_{k=1}^{p}\phi_k\,\ln(Gambling_{t-k})
+\sum_{k=1}^{p}\theta_k\,Unemp_{t-k}
+\delta_2 \ln(Pop_t)
+\delta_3 WPI_t
+\eta_t
$$

\begin{align}
H_0 &: \theta_1 = \theta_2 = \cdots = \theta_p = 0
&& \text{(Lagged unemployment has no predictive content for gambling)} \\
H_1 &: \exists\, k \in \{1,\dots,p\} \text{ such that } \theta_k \neq 0
&& \text{(Lagged unemployment has predictive content for gambling)}
\end{align}

In [ ]:
def granger_unemp_to_gambling_joint(
    ts_df,
    *,
    year_col=YEAR,
    maxlag=4,
    controls=( "log_pop", WPI)
):

    cols = [year_col, UNEMP, "log_gambling", *controls]
    d = ts_df[cols].dropna().sort_values(year_col).copy()

    rows = []

    for L in range(1, maxlag + 1):
        tmp = d.copy()

        # build lags
        for k in range(1, L + 1):
            tmp[f"g_L{k}"] = tmp["log_gambling"].shift(k)
            tmp[f"u_L{k}"] = tmp[UNEMP].shift(k)

        tmp = tmp.dropna()

        y = tmp["log_gambling"].astype(float)

        X = sm.add_constant(
            tmp[[f"g_L{k}" for k in range(1, L + 1)] +
                [f"u_L{k}" for k in range(1, L + 1)] +
                list(controls)]
            .astype(float)
        )

        res = sm.OLS(y, X).fit()

        # joint F-test: all unemployment lags = 0
        param_names = list(X.columns)
        R = np.zeros((L, len(param_names)))
        for i, k in enumerate(range(1, L + 1)):
            R[i, param_names.index(f"u_L{k}")] = 1.0

        ftest = res.f_test(R)

        # cumulative direction
        cum_coef = sum(res.params[f"u_L{k}"] for k in range(1, L + 1))

        rows.append({
            "Lag": L,
            "Cumulative unemployment effect": cum_coef,
            "Direction": "Positive" if cum_coef > 0 else "Negative",
            "F_stat": float(ftest.fvalue),
            "p_value": float(ftest.pvalue),
            "N": int(res.nobs)
        })

    return pd.DataFrame(rows)


In [ ]:
gd_nat = granger_unemp_to_gambling_joint(
    nat_all,
    year_col=YEAR,
    maxlag=5
)
print(gd_nat)


In [ ]:
region = df_flat[df_flat["State"] == "TAS"][
    [YEAR, UNEMP, "log_gambling", "log_pop", WPI]
]

gd_nsw = granger_unemp_to_gambling_joint(
    region,
    year_col=YEAR,
    maxlag=5
)
print(gd_nsw)


To summarise, we have two parts in our method.

Part $1$ tests for correlation using time-series OLS (national and regional) and panel regression (national).

Part $2$ tests for causation using Granger causality (national and regional).

Please tell me anything you think may be we should add or delete:), also here are some things to think about:


I am thinking about may be choose one from Times-series OLS at the national level and panel regression at the national level, maybe we should subtract the OLS at national level?
